In [1]:
import time
time_start = time.time()
import re
import os
import numpy as np
import scipy as sp
import mne
import nilearn.plotting as nplt
import nilearn.image as image
import pickle
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
from sklearn.metrics import roc_auc_score
from nodestimation.node_estimate import Node
from nodestimation.timewindow import TimeWindow, sliding_window
import nibabel as nib
import matplotlib.pyplot as plt
import nodestimation.project.path as path
from nodestimation.mneraw import\
    read_original_raw,\
    first_processing,\
    src_computation,\
    bem_computation,\
    read_original_trans,\
    forward_computation,\
    events_computation,\
    epochs_computation,\
    evokeds_computation,\
    noise_covariance_computation,\
    inverse_computation,\
    source_estimation,\
    coordinates_computation,\
    read_original_resec,\
    resection_area_computation,\
    connectivity_computation,\
    power_spectral_destiny_computation,\
    nodes_creation
from nodestimation.mlearning.features import \
    prepare_connectivity,\
    prepare_psd

subjects_dir, subjects = path.found_subject_dir()
tree = path.build_resources_tree(subjects)
# print(tree['B1C2'][1])
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
epochs_tmin, epochs_tmax = -15, 15
crop_time = 120
snr = 0.5  # use SNR smaller than 1 for raw data
lambda2 = 1.0 / snr ** 2
method = "sLORETA"
rfreq = 200
nfreq = 50
lfreq = 1
hfreq = 70
delta = (0.5, 4)
theta = (4, 7)
alpha = (7, 14)
beta = (14, 30)

# raw = None
# fp_raw = None
# bem = None
# src = None
# trans = None
# fwd = None
# events = None
# epochs = None
# cov = None
# evoked = None
# inv = None
# stc = None
# resec = None
# resec_mni = None
# labels_parc = None
# labels_aseg = None
# labels = None
# label_names = None
# lh_labels = None
# rh_labels = None
# label_ts = None

raw = read_original_raw('./', _subject_tree=tree['B1C2'])
fp_raw = first_processing(raw, lfreq, nfreq, hfreq, rfreq=rfreq, crop=crop_time, _subject_tree=tree['B1C2'])
bem = bem_computation('B1C2', subjects_dir, conductivity, _subject_tree=tree['B1C2'])
src = src_computation('B1C2', subjects_dir, bem, _subject_tree=tree['B1C2'])
trans = read_original_trans('./', _subject_tree=tree['B1C2'])
fwd = forward_computation(fp_raw, trans, src, bem, _subject_tree=tree['B1C2'])
events = events_computation(fp_raw, range(1, 59), [1 for i in range(58)], _subject_tree=tree['B1C2'])
epochs = epochs_computation(fp_raw, events, -1, +1, _subject_tree=tree['B1C2'])
cov = noise_covariance_computation(epochs, -1, 0, _subject_tree=tree['B1C2'])
evoked = evokeds_computation(epochs, _subject_tree=tree['B1C2'])
inv = inverse_computation(epochs.info, fwd[0], cov, _subject_tree=tree['B1C2'])
stc = source_estimation(epochs, inv, lambda2, method, _subject_tree=tree['B1C2'])
resec = read_original_resec('./', _subject_tree=tree['B1C2'])
resec_mni = resection_area_computation(resec, _subject_tree=tree['B1C2'])
labels_parc = mne.read_labels_from_annot('B1C2', parc='aparc', subjects_dir=subjects_dir)
labels_aseg = mne.get_volume_labels_from_src(src, 'B1C2', subjects_dir)
labels = labels_parc + labels_aseg
coords = coordinates_computation('B1C2', subjects_dir, labels,  _subject_tree=tree['B1C2'])
label_names = [label.name for label in labels]
lh_labels = [name for name in label_names if name.endswith('lh')]
rh_labels = [name for name in label_names if name.endswith('rh')]
label_ts = mne.extract_label_time_course(stc, labels, src[0], mode='mean_flip')
connectivity = connectivity_computation(label_ts, 200, [delta, theta, alpha, beta], ['coh', 'imcoh', 'plv', 'ppc', 'pli'], _subject_tree=tree['B1C2'])
psd = power_spectral_destiny_computation(epochs, inv, lambda2, [delta, theta, alpha, beta], method, 4, labels, _subject_tree=tree['B1C2'])
nodes = nodes_creation(
    labels,
    prepare_connectivity(label_names, connectivity),
    prepare_psd(label_names, psd),
    coords,
    resec_mni,
    _subject_tree=tree['B1C2']
)
print(nodes)
print('Runtime: ', time.time() - time_start)

Analysing project structure...
	Subject:  B1C2
		Trans file: ok
		Fwd file: ok
		Ave file: ok
		Stc file: ok
		Epo file: ok
		Src file: ok
		Bem file: ok
		Raw file: ok
		Coords file: ok
		Con file: ok
		Resec_mni file: ok
		Inv file: ok
		Cov file: ok
		Eve file: ok
		Psd file: ok
		Resec file: ok
	Files structure for B1C2 has been analysed. Files tree is built
Looking for original raw file in files tree...
Original raw file has been found; trying to read...
2 files found: ['./Source/Subjects/B1C2/B1C2_ii_run1_raw_tsss_mc_art_corr_raw.fif', './Source/Subjects/B1C2/B1C2_node_estimation_pipeline_file_first_processing_output_raw.fif']. Only the original files are going to be read
Opening raw data file ./Source/Subjects/B1C2/B1C2_ii_run1_raw_tsss_mc_art_corr_raw.fif...
    Range : 39000 ... 1257999 =     39.000 ...  1257.999 secs
Ready.
Successfully read
Looking for nepf raw file in files tree...
Nepf raw file has been found; trying to read...
2 files found: ['./Source/Subjects/B1C2/B1C2_

/home/ubuntu/PycharmProjects/NodesEstimation/nodestimation/project/path.py:138: RuntimeWarning: This filename (./Source/Subjects/B1C2/source_spaces_ico5.fif) does not conform to MNE naming conventions. All source space files should end with -src.fif, -src.fif.gz, _src.fif, _src.fif.gz, -fwd.fif, -fwd.fif.gz, _fwd.fif, _fwd.fif.gz, -inv.fif, -inv.fif.gz, _inv.fif or _inv.fif.gz
  read[type](tree[type][i])
/home/ubuntu/PycharmProjects/NodesEstimation/nodestimation/project/path.py:138: RuntimeWarning: This filename (./Source/Subjects/B1C2/source_spaces_oct5.fif) does not conform to MNE naming conventions. All source space files should end with -src.fif, -src.fif.gz, _src.fif, _src.fif.gz, -fwd.fif, -fwd.fif.gz, _fwd.fif, _fwd.fif.gz, -inv.fif, -inv.fif.gz, _inv.fif or _inv.fif.gz
  read[type](tree[type][i])
/home/ubuntu/PycharmProjects/NodesEstimation/nodestimation/project/path.py:138: RuntimeWarning: This filename (./Source/Subjects/B1C2/forward_ico5.fif) does not conform to MNE naming c

In [ ]:
fig, ax = plt.subplots()
ax.plot(psd['0.5-4Hz']['bankssts-lh']['freq'], psd['0.5-4Hz']['bankssts-lh']['psd'].mean(axis=0))
# ax.set_xlabel('Freq (Hz)')
# ax.set_xlim(psd['0.5-4Hz']['bankssts-lh']['freq'][[0, -1]])
# ax.set_ylabel('Power Spectral Density')

In [ ]:
print(connectivity['0.5-4Hz']['plv']['con'].shape)
print(connectivity['0.5-4Hz']['pli']['con'].shape)
print(connectivity['0.5-4Hz']['ppc']['con'].shape)
print(connectivity['0.5-4Hz']['coh']['con'].shape)
print(connectivity['0.5-4Hz']['imcoh']['con'].shape)
print(psd['0.5-4Hz']['bankssts-lh']['freq'].shape)

In [ ]:
con = connectivity['0.5-4Hz']['imcoh-based']['con'][:, :, 0]

con_s = np.zeros(con.shape)
for i in range(con.shape[0]):
    for j in range(con.shape[1]):
        con_s[j, i] = con[i, j]
# print('con')
# for i in range(con.shape[0]):
#     print(con[i, 0:10])
# print('con_s')
# for i in range(con.shape[0]):
#     print(con_s[i, 0:10])

plt.imshow(con)
plt.show()
plt.imshow(con.T)
plt.show()
plt.imshow(con + con_s)
plt.show()

In [ ]:
label_ypos_lh = list()

for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos_lh.append(ypos)

try:
    idx = label_names.index('Brain-Stem')

except ValueError:
    pass

else:
    ypos = np.mean(labels[idx].pos[:, 1])
    lh_labels.append('Brain-Stem')
    label_ypos_lh.append(ypos)


lh_labels = [label for (yp, label) in sorted(zip(label_ypos_lh, lh_labels))]

rh_labels = [label[:-2] + 'rh' for label in lh_labels
             if label != 'Brain-Stem' and label[:-2] + 'rh' in rh_labels]


node_colors = [label.color for label in labels]

node_order = lh_labels[::-1] + rh_labels

node_angles = mne.viz.circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) // 2])

In [ ]:
print(connectivity.keys())
print(connectivity['0.5-4Hz'].keys())
print(connectivity['0.5-4Hz']['coh-based'].keys())

In [ ]:
# for freq_band in connectivity:
#     for method in connectivity[freq_band]:
#         fig = plt.figure(num=None, figsize=(8, 8), facecolor='black')
#         mne.viz.plot_connectivity_circle(connectivity[freq_band][method]['con'][:, :, 0], label_names, n_lines=300,
#                      node_angles=node_angles, node_colors=node_colors,
#                      title='All-to-All Connectivity Epilepsy '
#                      'Condition ({} for {})'.format(method, freq_band), fig=fig)

methods = [
    'coh-based',
    'imcoh-based',
    'plv-based',
    'ppc-based',
    'pli-based'
]

freq_bands = [
    '0.5-4Hz',
    '4-7Hz',
    '7-14Hz',
    '14-30Hz'
]

for method in methods:
    for freq_band in freq_bands:
        fig = plt.figure(num=None, figsize=(25, 25), facecolor='black')
        mne.viz.plot_connectivity_circle(connectivity[freq_band][method]['con'][:, :, 0], label_names, n_lines=300,
                                         node_angles=node_angles, node_colors=node_colors,
                                         title='All-to-All Connectivity Epilepsy Condition ({} for {})'
                                         .format(method, freq_band), padding=8, fontsize_title=35, fontsize_colorbar=25,
                                         fontsize_names=20, fig=fig
                                         )

In [ ]:
del stc, src, raw, fwd,\
    bem, labels, label_ts, inv, cov
